In [ ]:
import pandas as pd
import numpy as np
import cv2
import math

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Lambda
from keras.layers import Input, ELU
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras import initializers
from keras.callbacks import ModelCheckpoint

In [ ]:
data_path = 'Data/SDC/training.csv'
data = pd.read_csv(data_path, header=None, skiprows=[0], names=['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed'])

In [ ]:
img_cols = 64
img_rows = 64
img_channels = 3

In [ ]:
def augment_image(image):
    image = cv2.cvtColor(image,cv2.COLOR_RGB2HSV)
    random_bright = .25 + np.random.uniform()
    image[:,:,2] = image[:,:,2] * random_bright
    image = cv2.cvtColor(image,cv2.COLOR_HSV2RGB)
    return(image)

In [ ]:
def preprocess_image(data):
    # Randomly pick left, center or right
    # camera image
    rand = np.random.randint(3)
    if (rand == 0):
        path_file = data['left'][0].strip()
        shift_ang = .25
    if (rand == 1):
        path_file = data['center'][0].strip()
        shift_ang = 0.
    if (rand == 2):
        path_file = data['right'][0].strip()
        shift_ang = -.25
    y = data['steering'][0] + shift_ang

    # Read image
    image = cv2.imread(path_file)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Crop image
    shape = image.shape
    image = image[math.floor(shape[0]/4):
    shape[0]-20, 0:shape[1]]

    # Resize image
    image = cv2.resize(image, (img_cols, img_rows),
    interpolation=cv2.INTER_AREA) 

    # Augment image
    image = augment_image(image)
    image = np.array(image)

    if np.random.choice([True, False]):
        image = cv2.flip(image,1)
        y = -y
    
    return(image, y)

In [ ]:
def batch_gen(data, batch_size):
    # Create empty numpy arrays
    batch_images = np.zeros((batch_size, img_rows,
    img_cols, img_channels))
    batch_steering = np.zeros(batch_size)

    small_steering_threshold = 0.8

    # Custom batch generator 
    while 1:
        for n in range(batch_size):
            i = np.random.randint(len(data))
            data_sub = data.iloc[[i]].reset_index()
            # Only keep training data with small 
            #steering angles with probablitiy 
            keep = False
            while keep == False:
                x, y = preprocess_image(data_sub)
                pr_unif = np.random
                if abs(y) < .01:
                    next;
                if abs(y) < .10:
                    small_steering_rand = np.random.uniform()
                    if small_steering_rand > 
                    small_steering_threshold:
                        keep = True
                else:
                    keep = True

            batch_images[n] = x
            batch_steering[n] = y
        yield batch_images, batch_steering

In [ ]:
input_shape = (img_rows, img_cols, img_channels)
model = Sequential()
model.add(Lambda(lambda x: x/255.-0.5, 
input_shape=input_shape))
model.add(Conv2D(3, (1, 1), padding='valid', kernel_initializer='he_normal'))
model.add(ELU())
model.add(Conv2D(32, (3, 3), padding='valid', kernel_initializer='he_normal'))
model.add(ELU())
model.add(Conv2D(32, (3, 3), padding='valid', kernel_initializer='he_normal'))
model.add(ELU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3, 3), padding='valid', kernel_initializer='he_normal'))
model.add(ELU())
model.add(Conv2D(64, (3, 3), padding='valid', kernel_initializer='he_normal'))
model.add(ELU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(128, (3, 3), padding='valid', kernel_initializer='he_normal'))
model.add(ELU())
model.add(Conv2D(128, (3, 3), padding='valid', kernel_initializer='he_normal'))
model.add(ELU())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(512, kernel_initializer='he_normal', activation='relu'))
model.add(ELU())
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer='he_normal'))
model.add(ELU())
model.add(Dropout(0.5))
model.add(Dense(16, kernel_initializer='he_normal'))
model.add(ELU())
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer='he_normal'))

opt = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=opt, loss='mse')
model.summary()

In [ ]:
callbacks = [ModelCheckpoint('checkpoints/sdc.h5', save_best_only=False)]

In [ ]:
train_generator = batchgen(data, batch_size)
val_generator = batchgen(data, batch_size)

In [ ]:
n_epochs = 8
batch_size = 64
steps_per_epoch = 500
validation_steps = 50

In [ ]:
history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=n_epochs, validation_data=val_generator, validation_steps=validation_steps, callbacks=callbacks)